<a href="https://colab.research.google.com/github/kwagdm/TIL/blob/master/pdf_download_final(%EB%8F%99%EB%AA%85)_7_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crawling

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
pdf_list = []
hw_list= []
date_list = []
for i in range(5, 33):
  url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(i)

  resp = requests.get(url)
  soup = BeautifulSoup(resp.content, 'lxml')
  total = soup.select('div.fileGoupBox ul li a')
  b= soup.select('span.date')
  count=0

  for idx,a in enumerate(total):
    if (a['href'][1]=='p')&(a['title'][-3:]=='hwp'):
      hw_list.append([b[count].text,a['title'],a['href']])
      count+=1
    elif (a['href'][1]=='p')&(a['title'][-3:]=='pdf'):
      pdf_list.append([b[count-1].text,a['title'],a['href']])

In [ ]:

pdf_list2 = []
title_list2 = []
date_list2 = []

for i in range(len(pdf_list)):
  pdf_list2.append(pdf_list[i][2])
  title_list2.append(pdf_list[i][1])
  date_list2.append(pdf_list[i][0][3:])

In [ ]:
coun=0
for i in range(len(pdf_list2)):
  url2 = 'http://www.bok.or.kr' + pdf_list2[i]
  if url2:
    coun+=1
  file_res = requests.get(url2)
  with open('{}\n{}'.format(date_list2[i], title_list2[i]), 'wb') as f:
    f.write(file_res.content)

# pdf to text


In [ ]:
!pip3 install pdfminer.six
from pdfminer.pdfinterp import PDFResourceManager,PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from io import open

     |████████████████████████████████| 5.6MB 2.6MB/s 
     |████████████████████████████████| 2.7MB 37.7MB/s 


In [ ]:
def convert_pdf_to_txt(title):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec = codec, laparams=laparams)
    fp = open('{}'.format(title), 'rb') # 여기에 저장된 url에 대한 response.content를 넣어줘야 함
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password = password, caching = caching, check_extractable=True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [ ]:
titles = []
for i in range(len(date_list2)):
  titles.append('{}\n{}'.format(date_list2[i], title_list2[i]))

txt_contents = []
for i in range(len(titles)):
  txt_contents.append(convert_pdf_to_txt(titles[i]))

In [ ]:
# 5페이지 범위 밖 파일들 삭제(2018년 파일들)
for i in range(7):
  del(txt_contents[0])


In [ ]:
#hwp 파일 하나 업로드 후 텍스트만 뽑기
txt_contents.append(convert_pdf_to_txt('2005.06.28 2005년 제11차 금통위 의사록 - PDF파일.pdf'))

# 날짜/문장 단위로 나눈 contents dataframe 만들기

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
#dataframe을 만들기 위해 회의 날짜를 크롤링 해오자
import re

title_date = []
for i in tqdm(range(5, 33)):
  url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(i)

  resp = requests.get(url)
  soup = BeautifulSoup(resp.content, 'lxml')
  date = soup.select('span.titlesub')
  title_date.append(re.findall('\)\((.+?)\)', str(date)))

title_only = []
for i in range(len(title_date)):
  for j in range(len(title_date[0])):
    title_only.append(title_date[i][j])

for i in range(7):
  del(title_only[0])

del(title_only[-8:])

len(title_only)

265

In [ ]:
len(title_only), len(txt_contents)

(265, 265)

In [ ]:
#contents 문장 단위로 나눠주기
text_sentence = []
for i in tqdm(range(len(txt_contents))):
  text_sentence.append(txt_contents[i])


# Section 나누기

In [ ]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'
    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [ ]:
import re 
def preprocess_minutes(minutes):
    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1
    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)
    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)
    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)
    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)
    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)
    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)
    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', "'Participants' Views", "'Government's View"]
    section_texts = (section2, section3)
    return  section_texts

In [ ]:
section_two_three = []
for i in tqdm(range(len(txt_contents))):
  section_two_three.append(preprocess_minutes(txt_contents[i]))

In [ ]:
section[:10]

[[],
 [],
 [],
 ['일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를  인상할 것으로 예견되고 실물경기도 견조한 회복세를 지속하고 있음에도 불구하  고 수익률곡선은 계속 평탄화되고 있다고 지적하였음.',
  '일각에서는 미국의 장기실  질균형금리 및 기대인플레이션의 하락을 그 원인으로 거론하고 있지만 이 같은  요인은 이미 이전부터 장기금리에 반영되어 왔으므로 최근의 수익률곡선 변화를  설명하는 데에 한계가 있는 것으로 보인다고 언급하였음.',
  '한편 이러한 현상을 향  후 실물경기의 침체 우려와 글로벌 장기채권 수요의 유입에 따른 결과로 해석하  는 시각도 있다고 소개하고, 이에 대한 관련부서의 견해를 물었음.',
  '이에 대해 관련부서에서는 수익률곡선의 평탄화나 장단기금리차의 역전이 장  기금리의 하락에 의해 주도되는 경우에는 향후 실물경기의 침체를 예고하는 신호  (signal)로 받아들여질 수 있겠지만, 최근 미국의 수익률곡선 평탄화는 주로 재무  부의 단기채권 발행 확대 방침, 미 연준의 금리인상 기대 등으로 단기금리가 크  게 상승한 데에 따른 것으로 보인다고 답변하였음.',
  '이에 동 위원은 미국의 세제개편안이 의회를 통과한다면 수익률곡선이나 정  책금리 경로가 가팔라질 가능성은 없는지 관련부서의 견해를 물었으며,  이에 대해 관련부서에서는 이번 세제개편안이 법인세율 인하 등 대규모 감세  안을 골자로 하고 있는 만큼 미 금융시장 지표가 동 법안의 처리 관측에 따라 등  락하는 양상을 보이고 있다고 설명하였음.',
  '비록 일부 선반영된 측면도 있겠지만  미 의회에서 세제개편안을 가결한다면 재정적자 보전을 위한 장기국채의 발행 전  망이 부각되면서 수익률곡선이 가팔라질 가능성도 있을 것으로 생각된다고 답변  - 9 -  \x0c이와 관련하여 동 위원은 한·미 간 장기금리의 동조화 경향에도 불구하고 우  리나라의 수익률곡선이 미국과 달리 가팔라지고 있는 이유는 무엇인지 관련부서  의 견해를 물었으며,

In [ ]:
section = []
for i in range(len(section_two_three)):
  section.append(section_two_three[i][0] + section_two_three[i][1])

In [ ]:
import pandas as pd
import numpy as np

dic_data = {'date': title_only, 'contents': section}
index = [i for i in range(len(txt_contents))]

df = pd.DataFrame(dic_data, index = index)
df

,date,contents
0,2017.12.28.,[]
1,2017.12.14.,[]
2,2017.12.7.,[]
3,2017.11.30.,[일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인...
4,2017.11.9.,[]
...,...,...
260,2005.8.11,[]
261,2005.7.7,[]
262,2005.6.23,[]
263,2005.6.9,[]


In [ ]:
df.to_csv("한국은행의사록.csv")

In [ ]:
df.iloc[3]['contents'][0]

'일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를  인상할 것으로 예견되고 실물경기도 견조한 회복세를 지속하고 있음에도 불구하  고 수익률곡선은 계속 평탄화되고 있다고 지적하였음.'